In [0]:
begin
  declare qry_str string;
  
  -- Build query string
  set qry_str = 
    "create or replace temporary view vw_src as "
    "select " || :select_list || " from " || :src_catalog || '.' || :src_schema || '.' || :src_table || " with(fetchSize = 100000) ";
  
  -- Add where clause
  if :watermark_col_type = 'timestamp' then
    set qry_str = concat(qry_str, " where " || :watermark_col_name || " > '" || :watermark_col_start_value || "' and "  || :watermark_col_name ||  " <= '" || :watermark_col_end_value || "'");
  else
    set qry_str = concat(qry_str, " where " || :watermark_col_name || " > " || :watermark_col_start_value || " and "  || :watermark_col_name ||  " <= " || :watermark_col_end_value);
  end if;
  
  -- Execute query
  execute immediate qry_str;
  select qry_str;
end

In [0]:
create or replace temporary function get_join_clause(primary_key string)
  returns string
  deterministic
  comment 'Converts a comma-separated list of primary key columns into a join clause'
  language python
  as $$
    pk_list = [col.strip() for col in primary_key.split(',')]
    join_clause = ' and '.join([f't.{col} = s.{col}' for col in pk_list])
    return join_clause
  $$;

In [0]:
begin
  declare qry_str string;
  declare join_clause string;
  
  -- Get join clause
  set join_clause = (select get_join_clause(:primary_key));
  
  -- Build merge statement
  set qry_str = 
    "merge into " || :sink_catalog || '.' || :sink_schema || '.' || :sink_table || " as t " ||
    "using vw_src as s " ||
    "on " || join_clause || " " ||
    "when matched then update set * " ||
    "when not matched then insert *";
    
  -- Execute merge statement
  execute immediate qry_str;
  select qry_str;
end